<a href="https://colab.research.google.com/github/Qadooshere/ML-Projects/blob/main/Rec_Bert4Rec_with_Review_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

## Import the necessary libraries


In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import ast
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader
from datasets import load_dataset

# Load Dataset

In [ ]:
# Retrieve the subset Beauty product from the Amazon Reviews dataset and store in CSVs in Drive folder "generate_embeddings" (folder must be created in Drive first)
dataset = load_dataset("amazon_us_reviews",'Beauty_v1_00')
for split, ds in dataset.items():
  ds.to_csv("/content/drive/MyDrive/bert4rec_generate_embeddings/my-dataset-beauty.csv", index=None)

In [2]:
# Load the dataset (first 100 rows to speed up)
reviews_df = pd.read_csv('/content/drive/MyDrive/bert4rec_generate_embeddings/my-dataset-beauty.csv', nrows = 100)  

In [3]:
newdf = reviews_df.copy()  # Create a copy of the existing DataFrame


In [5]:
newdf.shape

(100, 15)

## Define Pre-trained BERT Model and Tokenizer:

The code specifies the name of the pre-trained BERT model to be loaded. In this case, it is set to 'bert-base-uncased'.
The BertTokenizer.from_pretrained method is called to load the pre-trained tokenizer corresponding to the specified model name.
The BertModel.from_pretrained method is called to load the pre-trained BERT model corresponding to the specified model name.
The loaded BERT model is moved to the device specified earlier using .to(device)

In [4]:
# Set device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The conditional expression torch.device("cuda" if torch.cuda.is_available() else "cpu") creates a torch.device object that represents either the GPU (specified as "cuda") or the CPU (specified as "cpu") depending on the result of torch.cuda.is_available().

### Preprocessing Dataset

In [6]:
print(newdf.isnull().sum(),'/n')
print('Duplication :',newdf.duplicated().sum())
print(newdf.info())

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
dtype: int64 /n
Duplication : 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   marketplace        100 non-null    object
 1   customer_id        100 non-null    int64 
 2   review_id          100 non-null    object
 3   product_id         100 non-null    object
 4   product_parent     100 non-null    int64 
 5   product_title      100 non-null    object
 6   product_category   100 non-null    object
 7   star_rating        100 non-null    int64 
 8   helpful_votes      100 non-null    int64 
 9

## Define a function to generate review embeddings:
Tokenizes the text using the **BertTokenizer** and generates the embeddings using the **BertModel**
The with **torch.no_grad()** context manager is used to avoid storing intermediate values in memory, which saves memory usage

##Group the reviews by product ID and user ID, and generate embeddings for each group:


Groups the reviews in the reviews_df dataframe by the 
**product_id** and **customer_id** columns

Generates **review embeddings** for each review in the group using the get_review_embedding function defined earlier
Stores the review embeddings in a dictionary called **product_embeddings**

Modify the product_embeddings dictionary to store the embeddings for each user as a list of tuples: Replaces the list of embeddings for each user in the product_embeddings dictionary with a list of tuples containing the user ID and their embeddings.



In [5]:
def get_review_embedding(review_text):
    # Tokenize the text
    input_ids = torch.tensor([tokenizer.encode(str(review_text), truncation=True, max_length=256, add_special_tokens=True)]).to(device)  # Move input_ids to the GPU
    # Generate the embeddings
    with torch.no_grad():
        last_hidden_states = model(input_ids)[0]
    review_embedding = torch.mean(last_hidden_states, dim=1).squeeze().cpu().numpy()  # Move tensor to CPU and convert to NumPy array
    return review_embedding

In [6]:
# Group the reviews by product ID and user ID, and generate embeddings for each group
product_reviews = newdf.groupby(['product_id', 'customer_id'])['review_body'].apply(list)
product_embeddings = {}
for (product_id, user_id), reviews in product_reviews.items():
    review_embeddings = []
    for review in reviews:
        review_embedding = get_review_embedding(review)
        review_embeddings.append(torch.tensor(review_embedding).to(device))  # Move review embeddings to the GPU
    product_embeddings.setdefault(product_id, {}).setdefault(user_id, review_embeddings)

# Modify the product_embeddings dictionary to store the embeddings for each user as a list of tuples
for product_id, user_embeddings in product_embeddings.items():
    product_embeddings[product_id] = [(user_id, [embedding.to(device) for embedding in embeddings]) for user_id, embeddings in user_embeddings.items()]
# Create a list of embeddings in the desired format
product_embeddings_list = []

for product_id, user_embeddings in product_embeddings.items():
    user_embeddings_list = []

    for user_id, embeddings in user_embeddings:
        embeddings_np = [embedding.detach().cpu().numpy().tolist() for embedding in embeddings]
        user_embeddings_list.append((user_id, embeddings_np))

    product_embeddings_list.append([product_id, user_embeddings_list])

In [ ]:
product_embeddings_list[0]

In [10]:
product_embeddings_df = pd.DataFrame(product_embeddings_list, columns=['product_id', 'user_embeddings_list'])

In [11]:
newdf = pd.merge(newdf, product_embeddings_df, on='product_id', how='left')

In [22]:
newdf.shape

(100, 16)

In [12]:
newdf.head(2)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,user_embeddings_list
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0,0,0,1,Five Stars,"Love this, excellent sun block!!",2015-08-31,"[(1797882, [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0...."
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0,0,0,1,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31,"[(18381298, [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0..."


Converting review embeddings to tensors: The purpose of this code is to convert the review embeddings in the 'review_embeddings' column of df into PyTorch tensors. It first uses the apply() method with a lambda function to convert the string representation of the embeddings to a Python list using ast.literal_eval(), if the value is a string. Then, it applies another lambda function to convert each list of embeddings to a PyTorch tensor using torch.tensor(). The reshape(1, -1) call reshapes the tensor to have a single row and an inferred number of columns.

5. Normalizing review embeddings (optional):
The purpose of this code is to normalize the review embeddings in the 'review_embeddings' column of df
The fit_transform() method of the StandardScaler scales the embeddings using their mean and standard deviation. This code assumes that the 'review_embeddings' column contains numerical data.

Converting user_id and item_id to categorical variables: 
The purpose of this code is to convert the 'user_id' and 'item_id' columns in df into categorical variables. It uses the pd.Categorical function from the pandas library to convert the columns to a categorical data type. Categorical variables are used to represent discrete values with a limited number of unique values.

Mapping categorical variables to integer indices:
The purpose of this code is to map the categorical variables 'user_id' and 'item_id' in df to integer indices. It uses the cat.codes attribute of the categorical columns to assign a unique integer index to each unique category. The mapped integer indices are stored in new columns named 'user_index' and 'item_index' respectively. This mapping is useful when working with recommendation models that require integer indices for users and items instead of categorical labels.

In [7]:
# Convert review embeddings to tensors and reshape
for product_data in product_embeddings_list:
    user_embeddings = product_data[1]  # List of (user, embeddings) pairs
    for i in range(len(user_embeddings)):
        embeddings = user_embeddings[i][1]  # List of embeddings
        reshaped_embeddings = [torch.tensor(embedding).reshape(1, -1) for embedding in embeddings]
        user_embeddings[i] = (user_embeddings[i][0], reshaped_embeddings)


In [8]:
from sklearn.preprocessing import StandardScaler

# Normalize review embeddings
scaler = StandardScaler()
for product_data in product_embeddings_list:
    user_embeddings = product_data[1]  # List of (user, embeddings) pairs
    for i in range(len(user_embeddings)):
        embeddings = user_embeddings[i][1]  # List of embeddings
        normalized_embeddings = [scaler.fit_transform(embedding) for embedding in embeddings]
        user_embeddings[i] = (user_embeddings[i][0], normalized_embeddings)


In [13]:
user_ids = []
item_ids = []
user_indices = []
item_indices = []

# Iterate over the product_embeddings_list
for product_data in product_embeddings_list:
    user_embeddings = product_data[1]  # List of (user, embeddings) pairs
    for user_id, embeddings in user_embeddings:
        user_ids.append(user_id)
        item_ids.append(product_data[0])  # Product ID
        user_indices.append(len(user_ids) - 1)
        item_indices.append(len(item_ids) - 1)

# Assign the new values to the columns in the existing DataFrame
newdf['user_id'] = user_ids
newdf['item_id'] = item_ids
newdf['user_index'] = user_indices
newdf['item_index'] = item_indices


In [27]:
newdf.head(2)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,user_embeddings_list,user_id,item_id,user_index,item_index
0,US,1797882,R3I2DHQBR577SS,B001ANOOOE,2102612,The Naked Bee Vitmin C Moisturizing Sunscreen ...,Beauty,5,0,0,0,1,Five Stars,"Love this, excellent sun block!!",2015-08-31,"[(1797882, [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0....",30688607,B000094ZDX,0,0
1,US,18381298,R1QNE9NQFJC2Y4,B0016J22EQ,106393691,"Alba Botanica Sunless Tanning Lotion, 4 Ounce",Beauty,5,0,0,0,1,Thank you Alba Bontanica!,The great thing about this cream is that it do...,2015-08-31,"[(18381298, [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0...",12650290,B000FRWNL2,1,1


# Splitting the data into training and testing sets

The test_size parameter is set to 0.2, indicating that 20% of the data will be used for testing, and the remaining 80% will be used for training.
The **random_state** parameter is set to **42**, which is used as a seed value for the random number generator.  This ensures that the data will be split in the same way each time the code is executed, allowing for reproducibility of results.

In [14]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(newdf, test_size=0.2, random_state=42)  # Adjust the test_size as desired


In [ ]:
train_data

# Class definition: BERT4Rec



The purpose of this code is to define a PyTorch module called BERT4Rec for a recommendation model that incorporates BERT embeddings. 

**nn.Module Inheritance:**
The class BERT4Rec inherits from the nn.Module class, which is the base class for all neural network modules in PyTorch.

**Constructor**:

The __init__ method is the constructor of the BERT4Rec class.
Parameters:

**num_users**: The number of unique users in the recommendation system.

**num_items**: The number of unique items in the recommendation system.

**embedding_dim**: The dimensionality of the user and item embeddings.

**review_embedding_dim**: The dimensionality of the review embeddings.

**regularization_strength**: The strength of regularization to control overfitting.

###Attribute Initialization:

**user_embedding**: An embedding layer for users, using nn.Embedding with num_users as the number of unique users and embedding_dim as the embedding dimensionality.

**item_embedding**: An embedding layer for items, using nn.Embedding with num_items as the number of unique items and embedding_dim as the embedding dimensionality.

**item_transform**: A linear layer to transform the item embeddings to match the review_embedding_dim dimensionality.
fc: A linear layer that takes concatenated user and item embeddings as input and produces a single output for recommendation.

**review_embedding_dim**: Stores the value of 
review_embedding_dim for future reference within the class.

**regularization_strength**: Stores the value of regularization_strength for future reference within the class.

### Summary of Model Architecture:

The model architecture combines user and item embeddings with a review embedding to make recommendations.
The user and item embeddings are learned through the embedding layers (user_embedding and item_embedding).
The item embeddings are then transformed using the item_transform linear layer to match the review_embedding_dim dimensionality.
The transformed item embeddings and the user embeddings are concatenated.
The concatenated tensor is passed through the fc linear layer to produce a single output for recommendation.
The model aims to learn the relationships between users, items, and reviews to make accurate recommendations.







In [16]:
class BERT4Rec(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, review_embedding_dim, regularization_strength):
        super(BERT4Rec, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        self.item_transform = nn.Linear(embedding_dim, review_embedding_dim)  # Linear layer to transform item embedding size
        self.fc = nn.Linear(review_embedding_dim +embedding_dim, 1)
        self.review_embedding_dim = review_embedding_dim
        self.regularization_strength = regularization_strength

    def forward(self, user_indices, item_indices, review_embeddings):
        user_embedded = self.user_embedding(user_indices)
        item_embedded = self.item_embedding(item_indices)
        review_embeddings = review_embeddings.squeeze()
        review_embeddings = review_embeddings.type(torch.float32)
        # Regularization term
        item_transformed = self.item_transform(item_embedded)
        item_regularized = self.regularization_strength * review_embeddings
        item_regularized = item_transformed + item_regularized

        concatenated = torch.cat((user_embedded,item_regularized), dim=1)

        output = self.fc(concatenated) 
        return output

# Set hyperparameters
embedding_dim = 64
review_embedding_dim = 768  # Update the value according to the size of your review embeddings
learning_rate = 0.001
num_epochs = 15
batch_size = 32
regularization_strength = 0.003
num_items = len(newdf['item_index'].unique())
num_users = len(newdf['user_index'].unique())


# Instantiate the BERT4Rec model
model = BERT4Rec(num_users=num_users,
                 num_items=num_items,
                 embedding_dim=embedding_dim,
                 review_embedding_dim=review_embedding_dim,
                 regularization_strength=regularization_strength)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Convert tensors to PyTorch data types
train_user_indices = torch.LongTensor(train_data['user_index'].values)
train_item_indices = torch.LongTensor(train_data['item_index'].values)
# Convert review embeddings to tensors

train_review_embeddings = torch.stack([torch.tensor(embeddings[0][1]) for embeddings in train_data['user_embeddings_list'].values])

train_ratings = torch.FloatTensor(train_data['star_rating'].values)

# Create data loader for batching
train_dataset = torch.utils.data.TensorDataset(train_user_indices, train_item_indices, train_review_embeddings, train_ratings)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_user_indices, batch_item_indices, batch_review_embeddings, batch_ratings in train_loader:
        optimizer.zero_grad()
        outputs = model(user_indices=batch_user_indices, item_indices=batch_item_indices, review_embeddings=batch_review_embeddings)
        loss = criterion(outputs.squeeze(), batch_ratings)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * batch_user_indices.size(0)
    
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")


Epoch 1/15, Loss: 20.139391326904295
Epoch 2/15, Loss: 16.712335968017577
Epoch 3/15, Loss: 14.220412826538086
Epoch 4/15, Loss: 12.118376159667969
Epoch 5/15, Loss: 10.269171714782715
Epoch 6/15, Loss: 8.652211761474609
Epoch 7/15, Loss: 7.328273963928223
Epoch 8/15, Loss: 6.236619758605957
Epoch 9/15, Loss: 5.336564254760742
Epoch 10/15, Loss: 4.602384281158447
Epoch 11/15, Loss: 3.953199100494385
Epoch 12/15, Loss: 3.5007971048355104
Epoch 13/15, Loss: 3.0232475280761717
Epoch 14/15, Loss: 2.645865035057068
Epoch 15/15, Loss: 2.2982677936553957


##Method: forward
The forward method is the implementation of the forward pass for the BERT4Rec model. It defines how the input tensors are processed through the layers of the model to generate the output.



**Parameters**:

**user_indices**: Tensor containing the indices of the users for the current batch.

**item_indices**: Tensor containing the indices of the items for the current batch.

**review_embeddings**: Tensor containing the review embeddings for the current batch.

**Forward Pass:**
**user_embedded**: Embeds the user indices using the user_embedding layer.

**item_embedded**: Embeds the item indices using the item_embedding layer.

**review_embeddings**: Squeezes the review embeddings tensor to remove any unnecessary dimensions.

**review_embeddings**: Converts the data type of the squeezed review embeddings tensor to torch.float32.

**item_transformed**: Passes the item embeddings through the item_transform linear layer to match the review_embedding_dim dimensionality.

**item_regularized**: Computes the regularization term by multiplying the review embeddings by the regularization_strength and adding it to the transformed item embeddings.

**concatenated**: Concatenates the user embeddings and the regularized item embeddings along the second dimension (column-wise concatenation).

**output**: Passes the concatenated tensor through the fc linear layer to produce the final recommendation output.

### Summary of Forward() Method
This forward method defines the flow of data through the model's layers and computes the recommendation output. It is called automatically when the model is invoked with the input data during training or inference.

## Setting Hyperparameters:
**embedding_dim**: The dimensionality of the user and item embeddings. It is set to 64 in this example.

**review_embedding_dim**: The dimensionality of the review embeddings. This value needs to match the dimensionality of the review embeddings used in the model architecture.

**learning_rate**: The learning rate used by the optimizer during training. It is set to 0.001.

**num_epochs**: The number of training epochs, which determines how many times the model will iterate over the entire training dataset. It is set to 15.

**batch_size**: The number of samples per batch during training. It controls how many samples are processed together in each forward and backward pass. It is set to 32.

**regularization_strength**: The strength of regularization applied to the item embeddings. It controls the impact of the regularization term on the item embeddings during training. It is set to 0.003.

**num_items**: The number of unique items in the dataset. It is calculated based on the number of unique item indices in the 'item_index' column of the DataFrame.

**num_users**: The number of unique users in the dataset. It is calculated based on the number of unique user indices in the 'user_index' column of the DataFrame.


##Instantiating the BERT4Rec Model:

The code creates an instance of the BERT4Rec model by passing the necessary parameters to the constructor. The parameters include num_users, num_items, embedding_dim, review_embedding_dim, and regularization_strength that were defined earlier.


##Defining Loss Function and Optimizer:
The code defines the loss function (**criterion**) and the optimizer (**optimizer**) for training the model. The mean squared error (**nn.MSELoss**) is used as the loss function, and the Adam optimizer (**optim.Adam**) is used with the specified learning rate (**learning_rate**) and the model's parameters.



##Creating Data Loader for Batching:

The code creates a data loader (train_loader) that takes the training tensors (train_user_indices, train_item_indices, train_review_embeddings, train_ratings) and generates batches of data for training. The data loader is created using torch.utils.data.TensorDataset and torch.utils.data.DataLoader, and it shuffles the data (shuffle=True) to ensure randomization during training. The batch size is set to the specified batch_size

#Training Loop:



for epoch in range(num_epochs):

This loop iterates over each epoch, starting from 0 and up to num_epochs.
running_loss = 0.0:

This variable is used to keep track of the running loss for each epoch.
for batch_user_indices, batch_item_indices, batch_review_embeddings, batch_ratings in train_loader:

This loop iterates over each batch of training data generated by the train_loader.

The loop variables represent the user indices (batch_user_indices), item indices (batch_item_indices), review embeddings (batch_review_embeddings), and ratings (batch_ratings) for the current batch.

optimizer.zero_grad():

Clears the gradients of the model parameters before computing the gradients for the current batch.

**outputs** = model(user_indices=batch_user_indices, item_indices=batch_item_indices, review_embeddings=batch_review_embeddings):

Calls the forward method of the model to compute the recommendation outputs for the current batch of data.
Passes the batched user indices, item indices, and review embeddings as inputs to the model.

loss = criterion(outputs.squeeze(), batch_ratings):
Computes the loss between the recommendation outputs (outputs) and the batch ratings (batch_ratings).
The squeeze method is used to remove any unnecessary dimensions from the outputs tensor.

loss.backward():
Backpropagates the gradients of the loss through the model's parameters, computing the gradients for each parameter.

optimizer.step():
Updates the model's parameters based on the computed gradients using the specified optimizer (optimizer).

running_loss += loss.item() * batch_user_indices.size(0):
Updates the running loss by adding the current batch's loss multiplied by the batch size (batch_user_indices.size(0)).
This helps calculate the average loss per sample in the epoch.

epoch_loss = running_loss / len(train_data):
Computes the average loss per sample for the entire training dataset in the current epoch.

print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}"):
Prints the epoch number and the average loss per sample for the current epoch.


### Summary of Training Loop
The training loop iterates over the epochs and batches, performs forward and backward passes, updates the model's parameters, and computes the loss. At the end of each epoch, it prints the average loss for monitoring the training progress.

# Evaluation Metrics - MSE

In [20]:
# Convert tensors to PyTorch data types
test_user_indices = torch.LongTensor(test_data['user_index'].values)
test_item_indices = torch.LongTensor(test_data['item_index'].values)
test_review_embeddings = torch.stack([torch.tensor(embeddings[0][1]) for embeddings in test_data['user_embeddings_list'].values])
#test_review_embeddings = torch.stack(test_review_embeddings)
test_ratings = torch.FloatTensor(test_data['star_rating'].values)

# Calculate predictions
with torch.no_grad():
    test_outputs = model(test_user_indices, test_item_indices, test_review_embeddings)
    test_predictions = test_outputs.squeeze().cpu().numpy()

# Calculate evaluation metrics
# Example: Mean Squared Error (MSE)
mse = mean_squared_error(test_ratings, test_predictions)
print("Mean Squared Error:", mse)


Mean Squared Error: 20.666584


## Evaluation Score

###Converting Data to PyTorch Tensors:

Similar to the training data, the code converts the test data into PyTorch tensor format. The user indices (test_user_indices), item indices (test_item_indices), review embeddings (test_review_embeddings), and ratings (test_ratings) are converted to the appropriate tensor types (torch.LongTensor and torch.FloatTensor).

###Calculating Predictions:

The test data is passed through the trained model to generate predictions. The model is called with the test user indices, item indices, and review embeddings to obtain the recommendation outputs (test_outputs).
The squeeze method is applied to remove unnecessary dimensions from the outputs.
The predictions are then extracted from the tensors, converted to a NumPy array (test_predictions), and moved to the CPU.

###Calculating Evaluation Metrics:

In this example, the Mean Squared Error (MSE) metric is calculated as an evaluation metric.
The mean_squared_error function from the scikit-learn library is used to compute the MSE between the test ratings (test_ratings) and the predicted ratings (test_predictions).
The calculated MSE is stored in the variable mse.
Finally, the MSE value obtained is approximately 11.948236.